In [16]:
import cv2
import numpy as np
import torch

In [17]:
# Define range of blue color in HSV
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

In [18]:
path = "C:\\Users\\ukart\\Desktop\\Robomasters\\rmtest.mp4"
model_path = "C:\\Users\\ukart\\Desktop\\Robomasters\\best.pt"

In [19]:
def disp_image(im):
    cv2.imshow("image",im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [20]:
def blue_plate_detection(frame):
    
    frame = cv2.GaussianBlur(frame,(5,5),0)
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Create a binary image for blue color
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
        
    # Remove noise
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    # Find contours 
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw contours 
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
        
    return frame

## Sample frame output

In [21]:
# Load the video source
cap = cv2.VideoCapture(path)
cap.set(1, 600)
ret, frame = cap.read()

disp_image(blue_plate_detection(frame))

In [22]:
cap.release()
cv2.destroyAllWindows()

## Rewrite - Yolo

In [45]:
class Capture:
    # Constructor with depth camera
    def __init__(self, path):
        self.vidpath = path
        self.load_model()
        self.res = None

    

    def init_camera(self):
        if self.vidpath:
            self.cap = cv2.VideoCapture(path)
        else:
            self.cap = cv2.VideoCapture(0)
            

   
    def vid_capture(self, yolo=False):
        self.init_camera()
        
       
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            
            if self.res == None:
                self.res = list((frame.shape[:2]))
                self.res.reverse()
                
            if yolo == True:
                im = self.yolo_pipeline(frame)
            else:
                im = self.cv_pipeline(frame)
            
            # Display the output
            cv2.imshow("Video", im)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

        self.cap.release()
        cv2.destroyAllWindows()
        
    def load_model(self):
        self.model = torch.hub.load('ultralytics/yolov5', 'custom',path=model_path)
      

    # Process a color frame
    def cv_pipeline(self, frame):
         
        frame = cv2.GaussianBlur(frame,(5,5),0)

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create a binary image for blue color
        mask = cv2.inRange(hsv, lower_blue, upper_blue)

        # Remove noise
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        # Find contours 
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Draw contours 
        for contour in contours:
            x,y,w,h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)

        return frame
    
    def get_offset(self,x,y):
        
        res = self.res
        # calculate the center of the image
        img_cx = res[0] / 2
        img_cy = res[1] / 2

        # calculate the horizontal and vertical angles from the center of the image
        horizontal_angle = ((x - img_cx) / res[0]) * res[0]
        vertical_angle = ((y - img_cy) / res[1]) * res[1]
        
        return horizontal_angle, vertical_angle
        
    
    def yolo_pipeline(self, frame):
               
        results = self.model(frame)
        detections_rows = results.pandas().xyxy
        
        max_area = 0
        nearest_idx = -1
        rows = []
        for i in range(len(detections_rows)):
            rows = detections_rows[i].to_numpy()
            if(len(rows)) > 0 :
                x_min, y_min, x_max, y_max, conf, cls, label = rows[i]
                if conf < 0.5:
                    continue
                if (x_max-x_min)*(y_max-y_min) > max_area:
                    nearest_idx = i
                    max_area = (x_max-x_min)*(y_max-y_min)


            if nearest_idx != -1 and len(rows) > 0:
                x_min, y_min, x_max, y_max, conf, cls, label = rows[nearest_idx]                            
                ang_x, ang_y = self.get_offset( int((x_min+x_max)/2), int((y_min+y_max)/2) )
                label_conf = label + " " + str(round(conf,2)) + "|" + str(round(ang_x,2)) + "," + str(round(ang_y,2))

                cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
                cv2.putText(frame, label_conf, (int(x_min), int(y_min)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                
        return frame





In [46]:
cam = Capture(path)

Using cache found in C:\Users\ukart/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-2-6 Python-3.9.7 torch-1.8.0 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 1760518 parameters, 0 gradients, 4.2 GFLOPs
Adding AutoShape... 


In [47]:
# cam.vid_capture()

In [50]:
cam.vid_capture(yolo=True)